<center><b>DIGHUM101</b></center>
<center>5-4: Classification</center>

---

# Learning objectives

- Build a Logistic regression text classifier
- Cross-validation

In [ ]:
# Import libraries

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np 
import os
import nltk
nltk.download("movie_reviews")
from nltk.corpus import movie_reviews # Note we're importing the dataset here!
import pandas as pd
import re # regular expressions
import seaborn as sns
# CV (multiple train/test splitting)
from sklearn.model_selection import cross_val_score, train_test_split
# Regression
from sklearn.linear_model import LogisticRegression
# Tools to create our DTMs
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
# Speed up your machine learning setup
from sklearn.pipeline import Pipeline
# Mix up our training and test sets
from sklearn.utils import shuffle

import warnings
warnings.filterwarnings("ignore", category = DeprecationWarning)

# Classification of Textual Data

How can we translate this simple model of binary classification to text? Let's look at a corpus from `nltk` and build your own classifier using sklearn's machine learning `Pipeline`.

In [ ]:
# Extract our x (reviews) and y (judgements) variables
reviews = [movie_reviews.raw(fileid) for fileid in movie_reviews.fileids()]
judgements = [movie_reviews.categories(fileid)[0] for fileid in movie_reviews.fileids()]

In [ ]:
# Save in a dataframe
movies = pd.DataFrame({"Reviews" : reviews, 
                      "Judgements" : judgements})

In [ ]:
print(movies.shape)
movies.head()

As you might expect, this is a corpus of IMDB movie reviews. Someone went through and read each review, labeling it as either "positive" or "negative". The task we have before us is to create a model that can accurately predict whether a never-before-seen review is positive or negative. 

From the `movie_reviews` object let's take out the reviews and the judgement:

Let's look at a random review and its judgement:

In [ ]:
print("The human annotator's review was:", movies.Judgements[0])
print()
print(movies.Reviews[0][:500])

So right now we have a dataframe of movie reviews in the `Reviews` variable and a list of their corresponding judgements in the `Judgements` column. Awesome. What does this sound like to you? Independent and dependent variables? You'd be right!

`Reviews` is our x variable. `Judgements` is our y variable. Let's first reassign x and y for simplicity. While we're at it, we're going to set the random_state for our computer. Remember that this makes our result reproducible. We'll also `shuffle` so that we randomize the order of our observations, and when we split the testing and training data it won't be in a biased order. However, start learning about [stratified sampling](https://en.wikipedia.org/wiki/Stratified_sampling) and when you should use it instead of `shuffle`!

In [ ]:
x, y = shuffle(np.array(movies.Reviews), np.array(movies.Judgements), random_state = 1)

If you don't believe me that all we did is reassign and shuffle:

In [ ]:
x[0], print("Human annotator's review was:", y[0])

To get meaningful independent variables (words) we have to do some processing too (think DTM!). With `sklearn`'s text pipelines, we can quickly build a text classifier in only a few lines of Python: 

# Cross-validated pipepline

Remember training/test splitting? Lets do this `cv = 20` times! Read [here](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) for more info on cross-validation.

We should reasonably expect this to perform better than a single training/test split (see below).

In [ ]:
# Cross-validated model!
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                    ('tfidf', TfidfTransformer()),
                    ('clf', LogisticRegression(random_state = 0, penalty = "l2", C = 1000))
                     ])

scores = cross_val_score(text_clf, x, y, cv = 5)

print(scores, np.mean(scores))

The pipeline tells us three things happened:

1. `CountVectorizer`

2. `TfidfTransformer`

3. `LogisticRegression`

Let's walk through this step by step.

1. `CountVectorizer` does the same as before. It changes all the texts to quickly normalized words, and then simply counts the frequency of each word occuring in the corpus for each document. The feature array for each document at this point is simply the length of all unique words in a corpus, with the count for the frequency of each. This is the most basic way to provide features for a classifier - a document term matrix!

2. Remember that tfidf (term frequency inverse document frequency) is an algorithm that aims to find words that are important to specific documents. It does this by taking the term frequency (tf) for a specific term in a specific document, and multiplying it by the term's inverse document frequency (idf), which is the total number of documents divided by the number of documents that contain the term at least once. `TfidfTransformer` transforms the `CountVectorizer` into a tf-idf representation. 

A tfidf value is calculated for each term for each document. The feature arrays for a document is now the tfidf values. 

> Remember! The tfidf matrix is similar to our document term matrix, only now the values have been weighted according to their distribution across documents.

The pipeline now sends these tfidf feature arrays to 

3. `LogisticRegression`, what we learned from notebook 4-3. We add in an l2 penalization parameter because we have many more independent variables from our `dtm` than observations. 

Check out the [pipeline documentation here](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)

The code below breaks this down by each step, but combines the `CountVectorizer` and `TfidfTransformer` in the `TfidfVectorizer`.

In [ ]:
# Standard training/test split (no cross validation)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30, random_state = 0)

# get tfidf values
tfidf = TfidfVectorizer()
tfidf.fit(x)
x_train = tfidf.transform(x_train)
x_test = tfidf.transform(x_test)

# build and test logit
logit_class = LogisticRegression(random_state = 0, penalty = "l2", C = 1000, max_iter=1000)
model = logit_class.fit(x_train, y_train)
model.score(x_test, y_test)

# Important Features

After we train the model we can then index the tfidf matrix for the words with the most significant coefficients (remember, independent variables!) to get the most helpful features:

In [ ]:
feature_names = tfidf.get_feature_names_out()
top25pos = np.argsort(model.coef_[0])[-25:]
print("Top features for positive reviews:")
print(list(feature_names[j] for j in top25pos))
print()
print("Top features for negative reviews:")
top25neg = np.argsort(model.coef_[0])[:25]
print(list(feature_names[j] for j in top25neg))

# Prediction

We can also use our model to classify new reviews – all we have to do is extract the tfidf features from the raw text and send them to the model as our features (independent variables):

In [ ]:
new_bad_review = "This was the most awful worst super bad movie ever!"

tfidf_rev = tfidf.transform([new_bad_review])

model.predict(tfidf_rev)

In [ ]:
new_good_review = "This movie was great, awesome and good!"

tfidf_rev = tfidf.transform([new_good_review])

model.predict(tfidf_rev)

We could also have a look at the probabilities that our model is assigning to a new review.

In [ ]:
new_confusing_review = "I hated most things about it but it was okay"
# add "I guess" to the end of this sentence to see what happens

tfidf_rev = tfidf.transform([new_confusing_review])

print(model.classes_)
print(model.predict_proba(tfidf_rev))